## Connect to AzureML

First, we need to create an instance of MLClient to connect to Azure ML.

We are using `DefaultAzureCredential` to get access to workspace. `DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios.

Reference materials: [configure credential example](https://github.com/Azure/azureml-examples/blob/sdk-preview/sdk/jobs/configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [6]:
# authentication package
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [5]:
# handle to the workspace
from azure.ai.ml import MLClient

# get a handle to the workspace
ml_client = MLClient(
    subscription_id="40f5d89b-cca4-4f43-850b-46e831ef9da6",
    resource_group_name="Buy.Eng.Innovation.Omnicoding.AzureML",
    workspace_name="omnicoding",
    credential=credential,
)
gpu_cluster = None

# Provision the required resources

In [3]:
gpu_computes = {
    "fer-1xT4-cluster": "Standard_NC16as_T4_v3",
    "fer-4xT4-cluster": "Standard_NC64as_T4_v3",
    "fer-1xV100-cluster": "STANDARD_NC6s_v3",
    "fer-2xV100-cluster": "Standard_NC12s_v3",
}

In [8]:
from azure.ai.ml.entities import AmlCompute

gpu_compute_name = "fer-1xT4-cluster"
gpu_compute_size = gpu_computes[gpu_compute_name]

try:
    # let's see if the compute target already exists
    gpu_cluster = ml_client.compute.get(gpu_compute_name)
    print(
        f"You already have a cluster named {gpu_compute_name}, we'll reuse it as is."
    )
except Exception:
    print("Creating a new gpu compute target...")

    gpu_cluster = AmlCompute(
        name=gpu_compute_name,
        type="amlcompute",
        size=gpu_compute_size,
        min_instances=0,
        max_instances=100,
        idle_time_before_scale_down=180,
        tier="lowpriority",
    )

    gpu_cluster = ml_client.begin_create_or_update(gpu_cluster)

print(
    f"AMLCompute with name {gpu_cluster.name} is selected, the compute size is {gpu_cluster.size}"
)    

You already have a cluster named fer-1xT4-cluster, we'll reuse it as is.
AMLCompute with name fer-1xT4-cluster is selected, the compute size is STANDARD_NC16AS_T4_V3
